Import bibliotek

In [0]:
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum, avg, count, min, max, sum, lead, lag, first, last, row_number
import datetime


#Zadanie 3

Wykorzystaj przykłady z notatnika w SQL Windowed Aggregate Functions (cmd 11) i przepisz funkcje używając Spark API. 

In [0]:
transactions_data = [
    ( 1, '2011-01-01', 500),
( 1, '2011-01-15', 50),
( 1, '2011-01-22', 250),
( 1, '2011-01-24', 75),
( 1, '2011-01-26', 125),
( 1, '2011-01-28', 175),
( 2, '2011-01-01', 500),
( 2, '2011-01-15', 50),
( 2, '2011-01-22', 25),
( 2, '2011-01-23', 125),
( 2, '2011-01-26', 200),
( 2, '2011-01-29', 250),
( 3, '2011-01-01', 500),
( 3, '2011-01-15', 50 ),
( 3, '2011-01-22', 5000),
( 3, '2011-01-25', 550),
( 3, '2011-01-27', 95 ),
( 3, '2011-01-30', 2500)
]

transactions_columns = ["AccountId", "TranDate", "TranAmt"]
transactions_df = spark.createDataFrame(transactions_data, schema=transactions_columns)
transactions_df.createOrReplaceTempView("Transactions")


In [0]:
logical_data = [
(1,'George', 800),
(2,'Sam', 950),
(3,'Diane', 1100),
(4,'Nicholas', 1250),
(5,'Samuel', 1250),
(6,'Patricia', 1300),
(7,'Brian', 1500),
(8,'Thomas', 1600),
(9,'Fran', 2450),
(10,'Debbie', 2850),
(11,'Mark', 2975),
(12,'James', 3000),
(13,'Cynthia', 3000),
(14,'Christopher', 5000)
]

logical_columns = ["RowID", "FName", "Salary"]
logical_df = spark.createDataFrame(logical_data, schema=logical_columns)
logical_df.createOrReplaceTempView("Logical")


* Totals based on previous row

In [0]:
windowSpec = Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(Window.unboundedPreceding, Window.currentRow)

transactions_df_with_running_total = transactions_df.withColumn("RunTotalAmt", sum("TranAmt").over(windowSpec))

transactions_df_with_running_total.select("AccountId", "TranDate", "TranAmt", "RunTotalAmt").orderBy("AccountId", "TranDate").show()


+---------+----------+-------+-----------+
|AccountId|  TranDate|TranAmt|RunTotalAmt|
+---------+----------+-------+-----------+
|        1|2011-01-01|    500|        500|
|        1|2011-01-15|     50|        550|
|        1|2011-01-22|    250|        800|
|        1|2011-01-24|     75|        875|
|        1|2011-01-26|    125|       1000|
|        1|2011-01-28|    175|       1175|
|        2|2011-01-01|    500|        500|
|        2|2011-01-15|     50|        550|
|        2|2011-01-22|     25|        575|
|        2|2011-01-23|    125|        700|
|        2|2011-01-26|    200|        900|
|        2|2011-01-29|    250|       1150|
|        3|2011-01-01|    500|        500|
|        3|2011-01-15|     50|        550|
|        3|2011-01-22|   5000|       5550|
|        3|2011-01-25|    550|       6100|
|        3|2011-01-27|     95|       6195|
|        3|2011-01-30|   2500|       8695|
+---------+----------+-------+-----------+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.window import Window
import datetime

# Załóżmy, że SparkSession została już zainicjalizowana i nazywa się `spark`

# Przygotowanie danych
transactions_data = [
    (1, datetime.date(2011, 1, 1), 500.00),
    (1, datetime.date(2011, 1, 15), 50.00),
    (1, datetime.date(2011, 1, 22), 250.00),
    (1, datetime.date(2011, 1, 24), 75.00),
    (1, datetime.date(2011, 1, 26), 125.00),
    (1, datetime.date(2011, 1, 28), 175.00),
    (2, datetime.date(2011, 1, 1), 500.00),
    (2, datetime.date(2011, 1, 15), 50.00),
    (2, datetime.date(2011, 1, 22), 25.00),
    (2, datetime.date(2011, 1, 23), 125.00),
    (2, datetime.date(2011, 1, 26), 200.00),
    (2, datetime.date(2011, 1, 29), 250.00),
    (3, datetime.date(2011, 1, 1), 500.00),
    (3, datetime.date(2011, 1, 15), 50.00),
    (3, datetime.date(2011, 1, 22), 5000.00),
    (3, datetime.date(2011, 1, 25), 550.00),
    (3, datetime.date(2011, 1, 27), 95.00),
    (3, datetime.date(2011, 1, 30), 2500.00)
]

# Tworzenie DataFrame
transactions_columns = ["AccountId", "TranDate", "TranAmt"]
transactions_df = spark.createDataFrame(transactions_data, schema=transactions_columns)

# Specyfikacja okna dla sumy kumulatywnej
windowSpec = Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Obliczanie bieżącej sumy
transactions_df = transactions_df.withColumn("RunTotalAmt", sum("TranAmt").over(windowSpec))

# Wyświetlanie wyników
transactions_df.select("AccountId", "TranDate", "TranAmt", "RunTotalAmt").orderBy("AccountId", "TranDate").show()


+---------+----------+-------+-----------+
|AccountId|  TranDate|TranAmt|RunTotalAmt|
+---------+----------+-------+-----------+
|        1|2011-01-01|  500.0|      500.0|
|        1|2011-01-15|   50.0|      550.0|
|        1|2011-01-22|  250.0|      800.0|
|        1|2011-01-24|   75.0|      875.0|
|        1|2011-01-26|  125.0|     1000.0|
|        1|2011-01-28|  175.0|     1175.0|
|        2|2011-01-01|  500.0|      500.0|
|        2|2011-01-15|   50.0|      550.0|
|        2|2011-01-22|   25.0|      575.0|
|        2|2011-01-23|  125.0|      700.0|
|        2|2011-01-26|  200.0|      900.0|
|        2|2011-01-29|  250.0|     1150.0|
|        3|2011-01-01|  500.0|      500.0|
|        3|2011-01-15|   50.0|      550.0|
|        3|2011-01-22| 5000.0|     5550.0|
|        3|2011-01-25|  550.0|     6100.0|
|        3|2011-01-27|   95.0|     6195.0|
|        3|2011-01-30| 2500.0|     8695.0|
+---------+----------+-------+-----------+



In [0]:
windowSpec = Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(Window.unboundedPreceding, Window.currentRow)

transactions_df = transactions_df.withColumn("RunAvg", avg("TranAmt").over(windowSpec)) \
                                 .withColumn("RunTranQty", count("*").over(windowSpec)) \
                                 .withColumn("RunSmallAmt", min("TranAmt").over(windowSpec)) \
                                 .withColumn("RunLargeAmt", max("TranAmt").over(windowSpec)) \
                                 .withColumn("RunTotalAmt", sum("TranAmt").over(windowSpec))

transactions_df.select("AccountId", "TranDate", "TranAmt", "RunAvg", "RunTranQty", "RunSmallAmt", "RunLargeAmt", "RunTotalAmt") \
               .orderBy("AccountId", "TranDate").show()

+---------+----------+-------+------------------+----------+-----------+-----------+-----------+
|AccountId|  TranDate|TranAmt|            RunAvg|RunTranQty|RunSmallAmt|RunLargeAmt|RunTotalAmt|
+---------+----------+-------+------------------+----------+-----------+-----------+-----------+
|        1|2011-01-01|  500.0|             500.0|         1|      500.0|      500.0|      500.0|
|        1|2011-01-15|   50.0|             275.0|         2|       50.0|      500.0|      550.0|
|        1|2011-01-22|  250.0| 266.6666666666667|         3|       50.0|      500.0|      800.0|
|        1|2011-01-24|   75.0|            218.75|         4|       50.0|      500.0|      875.0|
|        1|2011-01-26|  125.0|             200.0|         5|       50.0|      500.0|     1000.0|
|        1|2011-01-28|  175.0|195.83333333333334|         6|       50.0|      500.0|     1175.0|
|        2|2011-01-01|  500.0|             500.0|         1|      500.0|      500.0|      500.0|
|        2|2011-01-15|   50.0|

* Calculating Totals Based Upon a Subset of Rows

In [0]:
windowSpec = Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(-2, 0)
windowSpecRowNumber = Window.partitionBy("AccountId").orderBy("TranDate")

transactions_df = transactions_df.withColumn("SlideAvg", avg("TranAmt").over(windowSpec)) \
                                 .withColumn("SlideQty", count("TranAmt").over(windowSpec)) \
                                 .withColumn("SlideMin", min("TranAmt").over(windowSpec)) \
                                 .withColumn("SlideMax", max("TranAmt").over(windowSpec)) \
                                 .withColumn("SlideTotal", sum("TranAmt").over(windowSpec)) \
                                 .withColumn("RN", row_number().over(windowSpecRowNumber))

transactions_df.select("AccountId", "TranDate", "TranAmt", "SlideAvg", "SlideQty", "SlideMin", "SlideMax", "SlideTotal", "RN") \
               .orderBy("AccountId", "TranDate").show()

+---------+----------+-------+------------------+--------+--------+--------+----------+---+
|AccountId|  TranDate|TranAmt|          SlideAvg|SlideQty|SlideMin|SlideMax|SlideTotal| RN|
+---------+----------+-------+------------------+--------+--------+--------+----------+---+
|        1|2011-01-01|  500.0|             500.0|       1|   500.0|   500.0|     500.0|  1|
|        1|2011-01-15|   50.0|             275.0|       2|    50.0|   500.0|     550.0|  2|
|        1|2011-01-22|  250.0| 266.6666666666667|       3|    50.0|   500.0|     800.0|  3|
|        1|2011-01-24|   75.0|             125.0|       3|    50.0|   250.0|     375.0|  4|
|        1|2011-01-26|  125.0|             150.0|       3|    75.0|   250.0|     450.0|  5|
|        1|2011-01-28|  175.0|             125.0|       3|    75.0|   175.0|     375.0|  6|
|        2|2011-01-01|  500.0|             500.0|       1|   500.0|   500.0|     500.0|  1|
|        2|2011-01-15|   50.0|             275.0|       2|    50.0|   500.0|    

* Logical Window


In [0]:
windowSpecRows = Window.orderBy("Salary").rowsBetween(Window.unboundedPreceding, Window.currentRow)

windowSpecRange = Window.orderBy("Salary").rangeBetween(Window.unboundedPreceding, Window.currentRow)

logical_df = logical_df.withColumn("SumByRows", sum("Salary").over(windowSpecRows)) \
                       .withColumn("SumByRange", sum("Salary").over(windowSpecRange))

logical_df.select("RowID", "FName", "Salary", "SumByRows", "SumByRange").orderBy("RowID").show()

+-----+-----------+------+---------+----------+
|RowID|      FName|Salary|SumByRows|SumByRange|
+-----+-----------+------+---------+----------+
|    1|     George|   800|      800|       800|
|    2|        Sam|   950|     1750|      1750|
|    3|      Diane|  1100|     2850|      2850|
|    4|   Nicholas|  1250|     4100|      5350|
|    5|     Samuel|  1250|     5350|      5350|
|    6|   Patricia|  1300|     6650|      6650|
|    7|      Brian|  1500|     8150|      8150|
|    8|     Thomas|  1600|     9750|      9750|
|    9|       Fran|  2450|    12200|     12200|
|   10|     Debbie|  2850|    15050|     15050|
|   11|       Mark|  2975|    18025|     18025|
|   12|      James|  3000|    21025|     24025|
|   13|    Cynthia|  3000|    24025|     24025|
|   14|Christopher|  5000|    29025|     29025|
+-----+-----------+------+---------+----------+



# Zadanie 4
Do tego notatnika dopisz użycie funkcji okienkowych LEAD, LAG, FIRST_VALUE, LAST_VALUE, ROW_NUMBER 

In [0]:
windowSpec = Window.orderBy("Salary")

logical_df = logical_df.withColumn("NextSalary", lead("Salary").over(windowSpec)) \
                       .withColumn("PreviousSalary", lag("Salary").over(windowSpec)) \
                       .withColumn("FirstSalary", first("Salary").over(windowSpec)) \
                       .withColumn("LastSalary", last("Salary").over(windowSpec)) \
                       .withColumn("RowNumber", row_number().over(windowSpec))

logical_df.select("RowID", "FName", "Salary", "NextSalary", "PreviousSalary", 
                  "FirstSalary", "LastSalary", "RowNumber").orderBy("RowID").show()

+-----+-----------+------+----------+--------------+-----------+----------+---------+
|RowID|      FName|Salary|NextSalary|PreviousSalary|FirstSalary|LastSalary|RowNumber|
+-----+-----------+------+----------+--------------+-----------+----------+---------+
|    1|     George|   800|       950|          null|        800|       800|        1|
|    2|        Sam|   950|      1100|           800|        800|       950|        2|
|    3|      Diane|  1100|      1250|           950|        800|      1100|        3|
|    4|   Nicholas|  1250|      1250|          1100|        800|      1250|        4|
|    5|     Samuel|  1250|      1300|          1250|        800|      1250|        5|
|    6|   Patricia|  1300|      1500|          1250|        800|      1300|        6|
|    7|      Brian|  1500|      1600|          1300|        800|      1500|        7|
|    8|     Thomas|  1600|      2450|          1500|        800|      1600|        8|
|    9|       Fran|  2450|      2850|          1600|  